<a href="https://colab.research.google.com/github/fairml-research/TEST/blob/main/DEFAULT_TRUE_S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
np.random.seed(7)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white", palette="muted", color_codes=True, context="talk")
from IPython import display
%matplotlib inline
import sys 
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import keras as ke
import keras.backend as K
from keras.layers import Input, Dense, Dropout
from keras.models import Model
# load ICU data set
#X, XNC, y, S, G = load_ICU_data('http://grarivincent.com/old/adult.data') 

In [3]:

def DEFAULTS(path):

    #column_names = ['age','job', 'marital', 'education', 'default', 'balance','housing', 'loan', 'contact','day','month','duration', 'campaign','pdays','previous', 'poutcome','y']
    input_data = (pd.read_csv(path, 
                              na_values="?", sep=r'\s*;\s*', engine='python',decimal=','))

    G = (input_data['MARRIAGE']-1).astype(str)
    G = G.pipe(pd.get_dummies, columns = ['MARRIAGE'])
    y = input_data['Y']
    S = input_data['SEX']-1
    X = (input_data
         .drop(columns=['Y','SEX','MARRIAGE'])
         .fillna('Unknown'))

    return X, y, S, G

In [4]:
X, y, S, G = DEFAULTS('http://grari.fr/Fairness/data/UCI_Credit_Card.csv')

In [5]:
# split into train/test set
X_train, X_test, y_train, y_test, S_train, S_test, G_train, G_test = train_test_split(X, y, S, G.values, test_size=0.2, random_state=7)

# standardize the data
scaler = MinMaxScaler().fit(X_train)
scale_df = lambda df, scaler: pd.DataFrame(scaler.transform(df), columns=df.columns, index=df.index)


X_train, X_test = X_train.pipe(scale_df, scaler), X_test.pipe(scale_df, scaler)

S_train, S_test =S_train.values, S_test.values
X_train, X_test =X_train.values, X_test.values

In [6]:
G

,-1.0,0.0,1.0,2.0
0,0,1,0,0
1,0,0,1,0
2,0,0,1,0
3,0,1,0,0
4,0,1,0,0
...,...,...,...,...
29995,0,1,0,0
29996,0,0,1,0
29997,0,0,1,0
29998,0,1,0,0


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
H = 15
H2 = 15
epochs_hgr=50
batch_size = 512
num_epochs = 25
learning_rate = 0.0001
batch_no = len(X_train) // batch_size
#X_train_inv = recon_X_batch_a.data.numpy()
#Y_train_inv = recon_Y_batch_a.data.numpy()
from sklearn.utils import shuffle
from torch.autograd import Variable
criterionMSE = nn.MSELoss()
import math
nb_a=200
from torch.nn import functional as F
def sigmoid(x):
  output = [1 / (1 + math.exp(-x)) for x in x]
  return output

class NN_b(nn.Module):
    def __init__(self):
        super(NN_b, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)        
    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        return x


model_Y_b = NN_b().cuda()
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer_Y = torch.optim.Adam(model_Y_b.parameters(), lr=0.001)
criterion_ADV =  nn.MSELoss()
#A0 = sensitive.mean() + sensitive.std()*Variable(torch.randn(data.shape[0]*nb_a, 1),requires_grad=False)
#recon_X_batch_a0, z_a0, recon_Y_batch_a0, mu_a0, logvar_a0 = model(data.repeat(nb_a,1).view(-1, 33),A0)
#recon_x, recon_xNC, S, U,  recon_y, mu, logvar = model(x_var, x_varNC, y_var,s_var)
from sklearn.metrics import accuracy_score
X_trainCUDA = Variable(torch.FloatTensor(X_train)).cuda()
y_trainCUDA = Variable(torch.FloatTensor(np.expand_dims(y_train,axis = 1))).cuda()

X_testCUDA = Variable(torch.FloatTensor(X_test)).cuda()
y_testCUDA = Variable(torch.FloatTensor(np.expand_dims(y_test,axis = 1))).cuda()


G_trainCUDA = Variable(torch.FloatTensor(G_train)).cuda()
G_testCUDA =  Variable(torch.FloatTensor(G_test)).cuda()

table = [0,0,0,0,0,0,0,0,0,0]
for epoch in range(num_epochs):
    #if epoch % 50 != 0:
        #x_train = Variable(torch.FloatTensor(X_train.values),requires_grad=True)
        #x_test = Variable(torch.FloatTensor(X_test.values),requires_grad=True)
        #print('Epoch {}'.format(epoch+1))
        
        #y_predX= model(x_train).data.numpy().T
        #y_predXinv= model(recon_X_batch_a0).data.numpy().T

        #accuracy = (y_train-np.squeeze(y_predX)).pow(2).sum()     
        #accuracycount = accuracy_score(Y_train_inv.squeeze(1), np.squeeze(y_predXinv))
        #print ("Train Accuracy:", accuracy)#,"Train Accuracy Count:", accuracycount, "CLP:", np.mean(logit_y_predX-logit_y_predXinv)**2)
    x_train, ytrain, sens_train, gtrain = shuffle(X_train, np.expand_dims(y_train,axis = 1),  S_train, G_train)

    #A = s_var.mean() + s_var.std()*Variable(torch.randn(batch_size*nb_a, 1),requires_grad=False)
    # Mini batch learning
    for i in range(batch_no):
        print('Epoch: ', epoch, 'Batch:',i)
        start = i * batch_size
        end = start + batch_size

        x_var = Variable(torch.FloatTensor(x_train[start:end])).cuda()
        y_var = Variable(torch.FloatTensor(ytrain[start:end])).cuda()
        s_var = Variable(torch.FloatTensor(sens_train[start:end])).cuda()
        g_train = Variable(torch.FloatTensor(gtrain[start:end])).cuda()

        optimizer_Y.zero_grad()
        ypred_var = model_Y_b(torch.cat([x_var.view(-1, x_var.shape[1])],1))
        lossY =criterion(ypred_var, y_var)
        betamse =0.45 #7   ## LAMBDA =0.7
        ret = betamse*torch.mean(lossY) -   torch.mean(ypred_var*g_train[:,2]) -   torch.mean(ypred_var*g_train[:,1])
        loss = lossY #+ret # + lossX_inv -2*lossX#+ betamse*MSEcount
        loss.backward()
        optimizer_Y.step()


        Ypred = torch.sigmoid(model_Y_b(torch.cat([X_trainCUDA.view(-1, X_trainCUDA.shape[1])],1)))>0.5
        Ho = (S_train==1)
        Fe = (S_train==0)
        odds=Ypred.float()[Ho].mean()/ Ypred.float()[Fe].mean()
        PRULE= min(odds, 1/odds)
        FP0=torch.abs(Ypred.float()[Ho & (y_trainCUDA.cpu().data.numpy().T==0)].mean()-Ypred.float()[Fe & (y_trainCUDA.cpu().data.numpy().T==0)].mean())
        FN1=torch.abs((1-Ypred.float()[Ho & (y_trainCUDA.cpu().data.numpy().T==1)].mean())-(1-Ypred.float()[Fe & (y_trainCUDA.cpu().data.numpy().T==1)].mean()))  

        Ypredtest = torch.sigmoid(model_Y_b(torch.cat([X_testCUDA.view(-1, X_testCUDA.shape[1] )],1)))>0.5
        Ho = (S_test==1)
        Fe = (S_test==0)
        odds=Ypredtest.float()[Ho].mean()/ Ypredtest.float()[Fe].mean()
        PRULEtest= min(odds, 1/odds)
        FP0test=torch.abs(Ypredtest.float()[Ho & (y_testCUDA.cpu().data.numpy().T==0)].mean()- Ypredtest.float()[Fe& (y_testCUDA.cpu().data.numpy().T==0)].mean())
        FN1test=torch.abs((1-Ypredtest.float()[Ho & (y_testCUDA.cpu().data.numpy().T==1)].mean())-(1-Ypredtest.float()[Fe & (y_testCUDA.cpu().data.numpy().T==1)].mean()))  

        print('ACC Train',(Ypred==y_trainCUDA).sum()/X_train.shape[0], 'PRULE',PRULE, 'ACC Test',(Ypredtest==y_testCUDA).sum()/X_test.shape[0],'PRULEtest',PRULEtest)

        table = np.vstack([table,[ (Ypred.cpu().data.numpy()==y_trainCUDA.cpu().data.numpy()).sum()/X_train.shape[0]*100,PRULE.cpu().data.numpy(),ret.cpu().data.numpy(), FP0,FN1,
                                  (Ypredtest.cpu().data.numpy()==y_testCUDA.cpu().data.numpy()).sum()/X_test.shape[0]*100,PRULEtest.cpu().data.numpy(),ret.cpu().data.numpy(),FP0test,FN1test]])

np.savetxt(sys.stdout, np.mean(table[1:,], axis=0).astype(float), '%5.2f')

Epoch:  0 Batch: 0
ACC Train tensor(0.7772, device='cuda:0') PRULE tensor(nan, device='cuda:0') ACC Test tensor(0.7850, device='cuda:0') PRULEtest tensor(nan, device='cuda:0')
Epoch:  0 Batch: 1
ACC Train tensor(0.7772, device='cuda:0') PRULE tensor(nan, device='cuda:0') ACC Test tensor(0.7850, device='cuda:0') PRULEtest tensor(nan, device='cuda:0')
Epoch:  0 Batch: 2
ACC Train tensor(0.7772, device='cuda:0') PRULE tensor(nan, device='cuda:0') ACC Test tensor(0.7850, device='cuda:0') PRULEtest tensor(nan, device='cuda:0')
Epoch:  0 Batch: 3
ACC Train tensor(0.7772, device='cuda:0') PRULE tensor(nan, device='cuda:0') ACC Test tensor(0.7850, device='cuda:0') PRULEtest tensor(nan, device='cuda:0')
Epoch:  0 Batch: 4
ACC Train tensor(0.7772, device='cuda:0') PRULE tensor(nan, device='cuda:0') ACC Test tensor(0.7850, device='cuda:0') PRULEtest tensor(nan, device='cuda:0')
Epoch:  0 Batch: 5
ACC Train tensor(0.7772, device='cuda:0') PRULE tensor(nan, device='cuda:0') ACC Test tensor(0.7850, 

KeyboardInterrupt: ignored

In [ ]:
y_trainCUDA.T==0

tensor([[True, True, True,  ..., True, True, True]], device='cuda:0')

In [ ]:
Ho & (ytrain.T==0)

array([[False,  True, False, ..., False,  True,  True]])

In [ ]:
ytrain

array([[1.],
       [0.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]])

In [ ]:
ytrain.shape

(24000, 1)

In [ ]:
Ho & (y_train==1)

9111     False
22157    False
8017     False
24099    False
12157    False
         ...  
5699     False
10742    False
16921    False
25796    False
28847    False
Name: Y, Length: 24000, dtype: bool

In [ ]:
g_train[:,3]

In [ ]:
g_train

HGR



# Variational Autoencoder

In [ ]:


import torch
import torch.nn as nn
import torch.nn.functional as F
H = 15
H2 = 15
epochs_hgr=50


table = [0,0,0,0,0,0,0,0,0,0,0,0]
batch_size = 512
num_epochs = 25
learning_rate = 0.0001
batch_no = len(X_train) // batch_size
#X_train_inv = recon_X_batch_a.data.numpy()
#Y_train_inv = recon_Y_batch_a.data.numpy()

from sklearn.utils import shuffle
from torch.autograd import Variable
criterionMSE = nn.MSELoss()
import math
nb_a=200
from torch.nn import functional as F
def sigmoid(x):
  output = [1 / (1 + math.exp(-x)) for x in x]
  return output

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)        
    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        return x

model_Y = NN().cuda()
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer_Y = torch.optim.Adam(model_Y.parameters(), lr=0.001)
criterion_ADV =  nn.MSELoss()
#A0 = sensitive.mean() + sensitive.std()*Variable(torch.randn(data.shape[0]*nb_a, 1),requires_grad=False)
#recon_X_batch_a0, z_a0, recon_Y_batch_a0, mu_a0, logvar_a0 = model(data.repeat(nb_a,1).view(-1, 33),A0)
#recon_x, recon_xNC, S, U,  recon_y, mu, logvar = model(x_var, x_varNC, y_var,s_var)
from sklearn.metrics import accuracy_score
X_trainCUDA = Variable(torch.FloatTensor(X_train)).cuda()
y_trainCUDA = Variable(torch.FloatTensor(np.expand_dims(y_train,axis = 1))).cuda()

X_testCUDA = Variable(torch.FloatTensor(X_test)).cuda()
y_testCUDA = Variable(torch.FloatTensor(np.expand_dims(y_test,axis = 1))).cuda()
for betamse0 in range(20):
    for o in range(10):
        for epoch in range(num_epochs):
            #if epoch % 50 != 0:
                #x_train = Variable(torch.FloatTensor(X_train.values),requires_grad=True)
                #x_test = Variable(torch.FloatTensor(X_test.values),requires_grad=True)
                #print('Epoch {}'.format(epoch+1))
                
                #y_predX= model(x_train).data.numpy().T
                #y_predXinv= model(recon_X_batch_a0).data.numpy().T

                #accuracy = (y_train-np.squeeze(y_predX)).pow(2).sum()     
                #accuracycount = accuracy_score(Y_train_inv.squeeze(1), np.squeeze(y_predXinv))
                #print ("Train Accuracy:", accuracy)#,"Train Accuracy Count:", accuracycount, "CLP:", np.mean(logit_y_predX-logit_y_predXinv)**2)
            x_train, ytrain, sens_train, gtrain = shuffle(X_train, np.expand_dims(y_train,axis = 1),  S_train, G_train)

            #A = s_var.mean() + s_var.std()*Variable(torch.randn(batch_size*nb_a, 1),requires_grad=False)
            # Mini batch learning
            for i in range(batch_no):
                print('Epoch: ', epoch, 'Batch:',i)
                start = i * batch_size
                end = start + batch_size

                x_var = Variable(torch.FloatTensor(x_train[start:end])).cuda()
                y_var = Variable(torch.FloatTensor(ytrain[start:end])).cuda()
                s_var = Variable(torch.FloatTensor(sens_train[start:end])).cuda()
                g_train = Variable(torch.FloatTensor(gtrain[start:end])).cuda()

                fx = model_Y_b(torch.cat([x_var.view(-1, x_var.shape[1])],1)).detach()

                optimizer_Y.zero_grad()
                bx = model_Y(torch.cat([x_var.view(-1, x_var.shape[1])],1))
                ypred_var = fx + (bx.T*(g_train[:,2]+g_train[:,1])).T
                lossY =criterion(ypred_var, y_var)
                betamse =betamse0/10 #7   ## LAMBDA =0.7
                ret = torch.mean(ypred_var) -   torch.mean(ypred_var*g_train[:,2]) -   torch.mean(ypred_var*g_train[:,1])
                loss = lossY +betamse*ret # + lossX_inv -2*lossX#+ betamse*MSEcount
                loss.backward()
                optimizer_Y.step()

                
                fxtrain = model_Y_b(torch.cat([X_trainCUDA.view(-1, x_var.shape[1])],1)).detach()
                bxtrain= model_Y(torch.cat([X_trainCUDA.view(-1, X_trainCUDA.shape[1])],1))
                Ypred =  torch.sigmoid(fxtrain + (bxtrain.T*(G_trainCUDA[:,0]+G_trainCUDA[:,1])).T)>0.5

                Ho = (S_train==1)
                Fe = (S_train==0)
                odds=Ypred.float()[Ho].mean()/ Ypred.float()[Fe].mean()
                PRULE= min(odds, 1/odds)
                FP0=torch.abs(Ypred.float()[Ho & (y_trainCUDA.cpu().data.numpy().T==0)].mean()-Ypred.float()[Fe & (y_trainCUDA.cpu().data.numpy().T==0)].mean())
                FN1=torch.abs((1-Ypred.float()[Ho & (y_trainCUDA.cpu().data.numpy().T==1)].mean())-(1-Ypred.float()[Fe & (y_trainCUDA.cpu().data.numpy().T==1)].mean()))  

                fxtest = model_Y_b(torch.cat([X_testCUDA.view(-1, x_var.shape[1])],1)).detach()
                bxtest = model_Y(torch.cat([X_testCUDA.view(-1, X_testCUDA.shape[1] )],1))
                Ypredtest =  torch.sigmoid(fxtest + (bxtest.T*(G_testCUDA[:,0]+G_testCUDA[:,1])).T)>0.5      
                Ho = (S_test==1)
                Fe = (S_test==0)
                odds=Ypredtest.float()[Ho].mean()/ Ypredtest.float()[Fe].mean()
                PRULEtest= min(odds, 1/odds)
                FP0test=torch.abs(Ypredtest.float()[Ho & (y_testCUDA.cpu().data.numpy().T==0)].mean()- Ypredtest.float()[Fe& (y_testCUDA.cpu().data.numpy().T==0)].mean())
                FN1test=torch.abs((1-Ypredtest.float()[Ho & (y_testCUDA.cpu().data.numpy().T==1)].mean())-(1-Ypredtest.float()[Fe & (y_testCUDA.cpu().data.numpy().T==1)].mean()))  

                print('ACC Train',(Ypred==y_trainCUDA).sum()/X_train.shape[0], 'PRULE',PRULE, 'ACC Test',(Ypredtest==y_testCUDA).sum()/X_test.shape[0],'PRULEtest',PRULEtest)

        table = np.vstack([table,[ num_epochs, betamse,(Ypred.cpu().data.numpy()==y_trainCUDA.cpu().data.numpy()).sum()/X_train.shape[0]*100,PRULE.cpu().data.numpy(),ret.cpu().data.numpy(), FP0,FN1,
                                  (Ypredtest.cpu().data.numpy()==y_testCUDA.cpu().data.numpy()).sum()/X_test.shape[0]*100,PRULEtest.cpu().data.numpy(),ret.cpu().data.numpy(),FP0test,FN1test]])

    np.savetxt('Default_ProxyFairness', table[1:,])

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Epoch:  20 Batch: 30
ACC Train tensor(0.8204, device='cuda:0') PRULE tensor(0.7975, device='cuda:0') ACC Test tensor(0.8185, device='cuda:0') PRULEtest tensor(0.7892, device='cuda:0')
Epoch:  20 Batch: 31
ACC Train tensor(0.8204, device='cuda:0') PRULE tensor(0.7975, device='cuda:0') ACC Test tensor(0.8185, device='cuda:0') PRULEtest tensor(0.7892, device='cuda:0')
Epoch:  20 Batch: 32
ACC Train tensor(0.8204, device='cuda:0') PRULE tensor(0.7975, device='cuda:0') ACC Test tensor(0.8185, device='cuda:0') PRULEtest tensor(0.7892, device='cuda:0')
Epoch:  20 Batch: 33
ACC Train tensor(0.8204, device='cuda:0') PRULE tensor(0.7975, device='cuda:0') ACC Test tensor(0.8185, device='cuda:0') PRULEtest tensor(0.7892, device='cuda:0')
Epoch:  20 Batch: 34
ACC Train tensor(0.8204, device='cuda:0') PRULE tensor(0.7975, device='cuda:0') ACC Test tensor(0.8185, device='cuda:0') PRULEtest tensor(0.7892, device='cuda:0')
Ep

In [ ]:
fxtrain.shape

In [ ]:
y_trainCUDA.shape

In [ ]:
g_train[:,1]

tensor([0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1.,
        1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1.,
        1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1.,
        1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
        1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
        0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1.,
        0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
        1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
        0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1.,
        0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1.,
        1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1.,
        0., 0., 0., 0., 1., 1., 1., 0., 

In [ ]:
(bx.T*g_train[:,0]).T

tensor([[-0.0000],
        [-0.0000],
        [-0.2023],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [ 0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.2143],
        [ 0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [ 0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [ 0.0000],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [-0.

In [ ]:
ypred_var

tensor([[-6.1044e+00],
        [ 1.9037e+05],
        [-2.5632e+00],
        [-5.1329e+00],
        [-1.7931e+00],
        [ 2.1200e+00],
        [-1.1092e+00],
        [ 5.5131e-01],
        [ 1.0300e+05],
        [ 4.7412e+05],
        [-1.1349e+00],
        [ 1.1263e+05],
        [ 2.5540e-01],
        [-5.9039e+00],
        [-2.5072e+00],
        [ 6.1787e+05],
        [ 6.6438e+05],
        [-2.6658e+00],
        [-6.2350e+00],
        [ 5.6209e+05],
        [ 1.0509e+00],
        [-2.9795e+00],
        [-2.9059e-01],
        [ 7.3794e-01],
        [ 2.7678e+05],
        [ 4.0200e-02],
        [ 1.9619e+05],
        [-6.2595e+00],
        [-2.5303e+00],
        [-2.5562e+00],
        [-2.6543e+00],
        [-9.8287e-02],
        [ 1.0517e+00],
        [-3.7581e+00],
        [ 1.1984e+05],
        [-6.3169e+00],
        [-4.8020e+00],
        [ 3.9099e+04],
        [-3.3419e+00],
        [-2.7430e+00],
        [ 4.3119e-01],
        [ 1.1492e+05],
        [ 8.3852e-01],
        [-1

In [ ]:
a=(bx.T*g_train[:,0]).T
a.shape

torch.Size([512, 1])

In [ ]:
fx.shape

torch.Size([512, 1])

In [ ]:
G_train.shape

(39072, 2)

In [ ]:
gtrain.shape

torch.Size([512, 2])